In [18]:
import numpy
import pandas as pd
def logfunc(s):
    is_log_on = False
    if is_log_on:
        print(s)

def compute_dps(intel, crit_score, hit_score, spellpower, haste):
    # print("Provided values : " + " " + str(intel) + " " +  str(crit_score) + " " +  str(hit_score) + " " +  str(spellpower))
    # Talents
    balance_of_power = 4 # +4% Hit
    focused_starlight = 4 # +4% crit for SF and Wrath
    moonkin_form = 5 # +5% Crit
    improved_mf = 10 # +10% Moonfire crit 
    starlight_wrath = True # reduce cast time by 0.5s
    vengeance = True # +100% Crit damange
    lunar_guidance = True # Spellpower bonus = 24% of total intel
    moonfury = 1.1 # +10% damage
    wrath_of_cenarius = 1.2 # +20% Spellpower for SF | +10% SpellPower for Wrath
    totem_of_wrath_hit = 0
    totem_of_wrath_crit = 0
    fight_length = 90 # in seconds

    # Talents Values
    if lunar_guidance:
        spellpower = spellpower + 0.24 * intel

    # Sets bonuses
    spellfire = True # SP bonus = +7% of total intellect
    spellstrike = True # 5% chance to have +92sp for 10s - No ICD
    windhawk = False # 8MP/5 KEK

    # Meta GEM - Chaotic Skyfire Diamond
    csd_equiped = False

    # Special Trinkets
    eye_of_mag = False # Grants 170 increased spell damage for 10 sec when one of your spells is resisted.
    silver_crescent = False # Use: Increases damage and healing done by magical spells and effects by up to 155 for 20 sec. (2 Min Cooldown)
    scryer_gem = False # Use: Increases spell damage by up to 150 and healing by up to 280 for 15 sec. (1 Min, 30 Sec Cooldown)
    quagmirran = False # Equip: Your harmful spells have a chance to increase your spell haste rating by 320 for 6 secs. (Proc chance: 10%, 45s cooldown)
    essence_sapphi = False #  Use: Increases damage and healing done by magical spells and effects by up to 130 for 20 sec. (2 Min Cooldown)

    # Translating stats to %
    # At level 70, 22.1 Spell Critical Strike Rating increases your chance to land a Critical Strike with a Spell by 1%
    # At level 70, 12.6 Spell Hit Rating increases your chance to Hit with Spells by 1%. Hit cap is 202 FLAT (not including talents & buffs).
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.

    # Moonfire base damage : 305 to 357 Arcane damage and then an additional 600 Arcane damage over 12 sec.
    MF_coeff = 0.15
    MF_coeff_dot = 0.52
    # Starfire base damage : 605 to 711 Arcane damage -> 658 on average
    SF_coeff = 1
    SF_average_damage = 658
    MF_average_damage = 331
    MF_average_dot_damage = 600
    partial_coeff = 0.5 # For the moment, let's say that in average, partials get 50% damage reduction
    sf_cast_time = 3
    sf_cast_time_ng = 2.5
    # Apply spell haste coefficients here

    # Spell power calculation for fight SP + lunar guidance 
    spellpower = spellpower + 0.24 * intel
    if spellfire:
        fight_spell_power = spellpower + 0.08 * intel

    # Hit chance
    # 12.6 Spell Hit Rating -> 1%
    hit_chance = min(99, 83 + (hit_score/12.6) + balance_of_power + totem_of_wrath_hit)
    logfunc("Hit chance is : " + str(hit_chance))

    # Crit chance
    # At level 70, 22.1 Spell Critical Strike Rating -> 1%
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.
    MF_crit_percent = crit_score/22.1 + intel/79.4 + improved_mf + moonkin_form + focused_starlight + totem_of_wrath_crit
    logfunc("Moonfire crit chance is : " + str(MF_crit_percent))
    SF_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight + totem_of_wrath_crit
    logfunc("Starfire crit chance is : " + str(SF_crit_percent))
    
    # Crit coeff
    if csd_equiped:
        crit_coeff = 2.09
    else:
        crit_coeff = 2
    
    # Spellstrike bonus:
    if spellstrike:
        spellstrike_bonus = 92
    else:
        spellstrike_bonus = 0
    
    # Prepare and launch the simulations
    loop_size = 100 # number of fights simulated
    average_dps = 0
    n = 0
    while n < loop_size:
        n = n +1
        # Initialization
        total_damage_done = 0
        fight_time = 0
        spellstrike_uptime = 0
        ff_uptime = 0
        mf_uptime = 0
        is_ff_up = False
        is_mf_up = False
        is_ng = False
        spellstrike_proc = False
        ng_proc = False
        # Time to kick ass and chew bubble gum
        while fight_time <= fight_length:
            loop_duration = 1 #GCD - can't be less, it's the rule !
            damage = 0
            if spellstrike_proc:
                fight_spell_power = spellpower + spellstrike_bonus
            else :
                fight_spell_power = spellpower
            # if FF not up, cast FF
            if not is_ff_up:
                logfunc("Casting Faerie Fire !")
                is_crit = False # can't crit on FF
                damage = 0 # and no damage applied
                if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                    is_hit = True
                    ff_uptime = 40
                    is_ff_up = True
                    # Test if spellstrike is proc
                    spellstrike_proc = (numpy.random.randint(1, high = 101, size = 1) <= 10)
                else:
                    is_hit = False
                    logfunc("Faerie Fire -> Resist !")
                loop_duration = 1 #GCD
            # if Moonfire not up, cast Moonfire
            else:
                if not is_mf_up:
                    logfunc("Casting Moonfire !")
                    loop_duration = 1 #GCD because we cast a spell
                    # Is it a hit ?
                    if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        is_hit = True
                        # Is it a crit ?
                        is_crit = (numpy.random.randint(1, high = 101, size = 1) <= MF_crit_percent)
                        # Is it a partial ?
                        if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                            damage = MF_average_damage + MF_coeff * fight_spell_power * partial_coeff
                        else:
                            damage = MF_average_damage + MF_coeff * fight_spell_power
                        # Apply damage
                        if is_crit:
                            damage = damage * crit_coeff
                        # DoT :
                        damage = damage + MF_average_dot_damage + (MF_coeff_dot * fight_spell_power * min(12, (fight_length - fight_time - 1))/12)
                        # There is a Hit ! update model
                        is_mf_up = True
                        mf_uptime = 12
                    else:
                        is_hit = False
                        logfunc("Moonfire -> Resist ! ")
                else:
                    # Cast Starfire
                    logfunc("Casting Starfire !")
                    # Is it a hit ?
                    if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        is_hit = True
                        # Is it a crit ?
                        is_crit = (numpy.random.randint(1, high = 101, size = 1) <= SF_crit_percent)
                        # Is it a partial ?
                        if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius * partial_coeff )) * moonfury
                        else:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury
                        if is_crit:
                            damage = damage * crit_coeff
                    else:
                        is_hit = False
                        logfunc("Starfire -> Resist ! ")
                    if is_ng:
                        loop_duration = sf_cast_time_ng
                    else:
                        loop_duration = sf_cast_time
                    is_ng = False # Consume NG once SF is cast

                # if there's a hit, we check Spellstrike proc

                # Update time and model
                fight_time = fight_time + loop_duration
                ff_uptime = ff_uptime - loop_duration
                mf_uptime = mf_uptime - loop_duration
                # Check the timer on buffs / debuffs
                spellstrike_uptime = spellstrike_uptime - loop_duration
                if spellstrike_uptime <= 0:
                    spellstrike_proc = False

                if mf_uptime <= 0:
                    is_mf_up = False
                if ff_uptime <= 0:
                    is_ff_up = False
                # @TODO if trinket available, activate   
                # Update nature's grace
                if is_crit:
                    is_ng = True
                total_damage_done = total_damage_done + damage

                # If there is a Hit, Check if spellstrike is proc or refreshed :
                if is_hit: 
                    if numpy.random.randint(1, high = 11, size = 1) == 10:
                        spellstrike_proc = True
                        spellstrike_uptime = 10
                        logfunc("Spellstrike proc !!!")

                # Print output
                logfunc("Loop Duration : " + str(loop_duration))
                logfunc("Loop Damage : " + str(damage))

        logfunc("Overall damage done : " + str(total_damage_done))
        #print("Overall DPS : "  + str(total_damage_done/fight_time)) # We use fight_time here in case SF lands after the fight_length mark
        average_dps = average_dps + (total_damage_done/fight_time)

    real_average_dps = average_dps / loop_size
    # print("Average DPS : " + str(real_average_dps))
    return(real_average_dps)

  


In [19]:
#compute_dps(intel = 359, crit_score = 269, hit_score = 52, spellpower = 1102, haste = 0)
import timeit
print("Starting loop")
# Generate the dataset
dataset_size = 1000 # number of rows
start = timeit.default_timer()


#for i in range(dataset_size): 
#    t_int = numpy.random.randint(300, high = 380, size = 1)
#    t_crit = numpy.random.randint(150, high = 280, size = 1)
#    t_hit = numpy.random.randint(110, high = 156, size = 1)
#    t_sp = numpy.random.randint(650, high = 1250, size = 1)
#    t_haste = numpy.random.randint(1, high = 11, size = 1)
#    t_dps = compute_dps(t_int, t_crit, t_hit,t_sp, t_haste)
#    df = df.append({"intel": t_int,
#               "crit": t_crit,
#               "hit": t_hit,
#               "sp": t_sp, 
#               "haste": t_haste,
#               "dps": t_dps}, ignore_index=True)

# Panda way of generating data
#t_int = numpy.random.randint(300, high = 380, size = dataset_size)
#t_crit = numpy.random.randint(150, high = 280, size = dataset_size)
#t_hit = numpy.random.randint(110, high = 156, size = dataset_size)
#t_sp = numpy.random.randint(650, high = 1250, size = dataset_size)
#t_haste = numpy.random.randint(1, high = 11, size = dataset_size)
v_compute_dps = numpy.vectorize(compute_dps)
df = pd.DataFrame(dict(
    intel = numpy.random.randint(300, high = 450, size = dataset_size),
    crit = numpy.random.randint(150, high = 320, size = dataset_size),
    hit = numpy.random.randint(110, high = 156, size = dataset_size),
    sp = numpy.random.randint(650, high = 1400, size = dataset_size),
    haste = numpy.random.randint(1, high = 11, size = dataset_size)
))
print('Dataframe generated, adding dps for each row.')
# df = pd.DataFrame(t_int, t_crit, t_hit, t_sp, t_haste, columns=["intel", "intel", "hit", "sp", "haste"])
df['dps'] = v_compute_dps(df.intel, df.crit, df.hit, df.sp, df.haste)
stop = timeit.default_timer()
print("End of loop. Duration: ", stop - start )  
df.to_csv(path_or_buf='C:/path/to/your/Jupyter/output/dps_generated-2.csv')

Starting loop
Dataframe generated, adding dps for each row.
End of loop. Duration:  320.4079637999821


In [38]:
df.to_csv(path_or_buf='C:/path/to/your/Jupyter/output/dps_generated.csv')

In [14]:
n = 0
all_dps = 0
loop_size = 50
while n < loop_size :
    n = n + 1
    # all_dps = all_dps + compute_dps(intel = 364, crit_score = 229, hit_score = 167, spellpower = 1179, haste = 0)
    all_dps = all_dps + compute_dps(intel = 381, crit_score = 243, hit_score = 135, spellpower = 1162, haste = 0)
average_dps = all_dps / loop_size
print("Average DPS is : ", average_dps)

Average DPS is :  798.2039028254513
